# Pytorch入门实战（7）：基于BERT实现简单的中文文本摘要任务（Summarization task）

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iioSnail/chaotic-transformer-tutorials/blob/master/bert_summarization_demo.ipynb)

In [1]:
# 如果你没有使用Google Drive，请不要运行这个代码块
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# 本文涉及知识点

1. [nn.Transformer的使用](https://blog.csdn.net/zhaohongfei_358/article/details/126019181)
2. [Transformer源码解读](https://blog.csdn.net/zhaohongfei_358/article/details/126085246) (了解即可)
3. [Pytorch中DataLoader和Dataset的基本用法](https://blog.csdn.net/zhaohongfei_358/article/details/122742656)
4. [Masked-Attention的机制和原理](https://blog.csdn.net/zhaohongfei_358/article/details/125858248)
5. [Pytorch自定义损失函数](https://blog.csdn.net/zhaohongfei_358/article/details/125759911)
6. [Hugging Face快速入门](https://blog.csdn.net/zhaohongfei_358/article/details/126224199)

# 本文内容

本文将使用Hugging Face提供的Bert模型和数据集进行迁移学习，完成购物评论的中文文本摘要任务（Summarization Task）。最终效果为：

原评论：本人账号被盗，资金被江西（杨建）挪用，请亚马逊尽快查实，将本人的200元资金退回。本人已于2017年11月30日提交退货申请，为何到2018年了还是没解决？亚马逊是什么情况？请给本人一个合理解释。
摘要后：此书不是本人购买

Hugging Face虽然提供了Summarization任务很方便的迁移学习API，但本文并不会使用。为了更好的知识学习和泛化，本文将会采用一种更为通用的方式来模型构造和模型训练，所以本文采用的为基础的中文bert模型`bert-base-chinese`。


# 环境配置

本文重点依赖Hugging Face的两个重要类库datasets和transformers，所以需要安装：

```
transformers==4.21
datasets==2.4
```

In [ ]:
!pip install datasets
!pip install transformers

导入本文要使用的所有依赖包:

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# 用于加载hugging face数据集
from datasets import load_dataset
# 用于加载bert-base-chinese模型的分词器
from transformers import AutoTokenizer
# 用于加载bert-base-chinese模型
from transformers import AutoModel
from pathlib import Path

# 全局配置

定义一些全局变量，我是不太喜欢一些全局变量在函数中传来传去的，太麻烦了。

In [3]:
batch_size = 16
# 文本（评论）的最大长度
text_max_length = 512
# 摘要的最大长度
summary_max_length = 48
epochs = 1000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 每多少步，打印一次loss
log_per_step = 1
# 每多少步存储一次模型
save_per_step = 5000

# 模型存储路径
model_dir = Path("./drive/MyDrive/model/transformer_checkpoints")
# 如果工作目录不存在，则创建一个
os.makedirs(model_dir) if not os.path.exists(model_dir) else ''

print("Device:", device)

Device: cpu


# 数据处理

## 加载数据集

在Hugging Face中找了一圈，最终锁定了一个叫`amazon_reviews_multi`的数据集（[链接](https://huggingface.co/datasets/amazon_reviews_multi/viewer/zh/train)）：

<img src="./images/hf_6.png" width="1000">

本次由于是做文本摘要，所以只需要review_body作为评论内容和review_title作为摘要内容。

我们先来加载一下数据集：

In [4]:
dataset = load_dataset("amazon_reviews_multi", "zh")

Reusing dataset amazon_reviews_multi (C:\Users\zhaohongfei1\.cache\huggingface\datasets\amazon_reviews_multi\zh\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

加载成功后，来看一下内容：

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

可以看到该数据集提供了train/validation/test三份，为了简单起见，我们不使用validation和test数据集。

## Dataset And Dataloader

加载好数据集后，我们就可以开始构建Dataset了，我们这里Dataset就是返回评论和其摘要：

In [6]:
class SummarizationDataset(Dataset):

    def __init__(self, mode='train'):
        super(SummarizationDataset, self).__init__()
        # 拿到对应的数据
        self.dataset = dataset[mode]

    def __getitem__(self, index):
        # 取第index条
        data = self.dataset[index]
        # 取其评论
        text = data['review_body']
        # 取对应的摘要
        summary = data['review_title']
        # 返回
        return text, summary

    def __len__(self):
        return 16
        # return len(self.dataset)

In [7]:
train_dataset = SummarizationDataset()

我们来打印看一下；

In [8]:
train_dataset.__getitem__(0)

('本人账号被盗，资金被江西（杨建）挪用，请亚马逊尽快查实，将本人的200元资金退回。本人已于2017年11月30日提交退货申请，为何到2018年了还是没解决？亚马逊是什么情况？请给本人一个合理解释。',
 '此书不是本人购买')

构造好Dataset后，就可以来构造Dataloader了。在构造Dataloader前，我们需要先定义好分词器：

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

我们来尝试使用一下分词器：

In [10]:
tokenizer("我正在学习深度学习", return_tensors='pt')

{'input_ids': tensor([[ 101, 2769, 3633, 1762, 2110,  739, 3918, 2428, 2110,  739,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

可以正常运行。其中101表示“开始”(`[CLS]`)，102表示句子结束(`[SEP]`)

我们接着构造我们的Dataloader。我们需要定义一下collate_fn，在其中完成对句子进行编码、填充、组装batch等动作：

In [11]:
def collate_fn(batch):
    """
    将一个batch的文本句子转成tensor，并组成batch。
    :param batch: 一个batch的句子，例如: [('评论', '摘要'), ('评论', '摘要'), ...]
    :return: 处理后的结果，例如：
             src: {'input_ids': tensor([[ 101, ..., 102, 0, 0, ...], ...]), 'attention_mask': tensor([[1, ..., 1, 0, ...], ...])}
             tgt和tgt_y与src格式一样
             n_tokens为本轮预测时有效token数
    """
    text, summary = zip(*batch)
    text, summary = list(text), list(summary)

    # src是要送给bert的，所以不需要特殊处理，直接用tokenizer的结果即可
    # padding='max_length' 不够长度的进行填充
    # truncation=True 长度过长的进行裁剪
    src = tokenizer(text, padding='max_length', max_length=text_max_length, return_tensors='pt', truncation=True)
    tgt = tokenizer(summary, padding='max_length', max_length=summary_max_length, return_tensors='pt', truncation=True)

    tgt_y = {}
    for key, value in tgt.items():
        tgt_y[key] = value[:, 1:]

    for key, value in tgt.items():
        tgt[key] = value[:, :-1]

    n_tokens = tgt_y['attention_mask'].sum().item()

    return src, tgt, tgt_y, n_tokens

In [12]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

我们来看一眼train_loader的数据：

In [13]:
next(iter(train_loader))[0]

{'input_ids': tensor([[ 101, 3119, 1168,  ...,    0,    0,    0],
        [ 101, 7490, 2094,  ...,    0,    0,    0],
        [ 101, 7509, 5056,  ...,    0,    0,    0],
        ...,
        [ 101, 4263, 6564,  ...,    0,    0,    0],
        [ 101, 6579,  743,  ...,    0,    0,    0],
        [ 101,  741, 7027,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

# 构建模型

这里我们使用bert作为encoder，然后使用`nn.TransformerDecoder`作为Decoder，然后再加上最后一个预测层组成完整的模型，如下图所示：

<img src="./images/bert_transformer.png" width="400">

In [14]:
class SummarizationModel(nn.Module):

    def __init__(self):
        super(SummarizationModel, self).__init__()

        # 加载bert模型
        self.bert = AutoModel.from_pretrained("bert-base-chinese")
        # 定义Decoder层
        decoder_layer = nn.TransformerDecoderLayer(d_model=768, nhead=8, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)

        # 从bert中把embedding层提取出来，因为pytorch的Decoder中不包含embedding部分
        self.embeddings = self.bert.embeddings
        # 最后的预测层
        self.predictor = nn.Linear(768, tokenizer.vocab_size)

    def forward(self, src, tgt):
        """
        前向传播，获取decoder的输出。注意是decoder的输出，不是最后线性层的输出
        :param src: 分词后的评论数据
        :param tgt: 前面累计预测出的结果
        :return: decoder的输出
        """
        # 将src直接序列解包传入bert，因为bert和tokenizer是一套的，所以可以这么做。
        # 得到encoder的输出
        last_hidden_state = self.bert(**src).last_hidden_state
        # 构造tgt_mask，就是那个阶梯形状的mask
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt['input_ids'].size(-1)).to(device)
        # 构造key_padding_mask，用于mask非句子成分
        tgt_key_padding_mask = tgt['attention_mask'] == 0
        # 将tgt的tensor提取出来，作为decoder的输入。
        decoder_inputs = self.embeddings(tgt['input_ids'])
        # 将encoder的输出和tgt作为decoder的输入传入decoder，得到输出
        decoder_outputs = self.decoder(tgt=decoder_inputs, memory=last_hidden_state, tgt_mask=tgt_mask, tgt_key_padding_mask=tgt_key_padding_mask)

        return decoder_outputs

In [15]:
model = SummarizationModel()
model = model.to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


定义好模型后，我们来定义一下损失函数，由于需要稍微做一些特殊处理，所以我单独写了个类：

In [16]:
class SummarizationLoss(nn.Module):

    def __init__(self):
        super(SummarizationLoss, self).__init__()
        # 使用经典的多分类CrossEntropyLoss作为损失函数，忽略index=0的，因为他们是填充
        self.criteria = nn.CrossEntropyLoss(ignore_index=0)

    def forward(self, outputs, tgt_y, n_tokens):
        """
        损失函数的前向传递
        :param outputs: 最终预测层的输出。例如，Shape为(64, 47, 768)，表示64个句子，每个句子47个词，每个词768维
        :param tgt_y: Label。例如，Shape为(64, 47)，表示64个句子，每个句子47个词
        :param n_tokens: 有效词的数量（非填充词的数量）。例如，1283表示在这64*47个词中，有1283个有效词
        :return: loss
        """
        # 由于有多个句子构成，每个句子有多个词，所以flatten一下，将Shape变成(64*47)
        targets = tgt_y['input_ids'].flatten()
        # outputs同理，将前面两个维度合并
        outputs = outputs.view(-1, tokenizer.vocab_size)
        # 计算损失，然后正则化一下，就是平均一下，平均到每个词上的损失
        return self.criteria(outputs, targets) / n_tokens

# 训练模型

接下来开始正式训练模型，首先定义出损失函数和优化器：

In [17]:
criteria = SummarizationLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

开始训练：

In [ ]:
# 首先将模型调成训练模式
model.train()

# 清空一下cuda缓存
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 定义几个变量，帮助打印loss
total_loss = 0.
# 记录步数
step = 0

# 由于src，tgt都是字典类型的，定义一个辅助函数帮助to(device)
def to_device(dict_tensors):
    result_tensors = {}
    for key, value in dict_tensors.items():
        result_tensors[key] = value.to(device)
    return result_tensors

# 开始训练
for epoch in range(epochs):
    for i, (src, tgt, tgt_y, n_tokens) in enumerate(train_loader):
        # 从batch中拿到训练数据
        src, tgt, tgt_y = to_device(src), to_device(tgt), to_device(tgt_y)
        # 传入模型进行前向传递
        outputs = model(src, tgt)
        # 将decoder的输出送给predictor进行预测
        outputs = model.predictor(outputs)
        # 计算损失
        loss = criteria(outputs, tgt_y, n_tokens)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += float(loss)
        step += 1

        if step % log_per_step == 0:
            print("Epoch {}/{}, Step: {}/{}, total loss:{:.4f}".format(epoch+1, epochs, i, len(train_loader), total_loss))
            total_loss = 0


        if step % save_per_step == 0:
            torch.save(model, model_dir / f"model_{step}.pt")

        del src, tgt, tgt_y, outputs

Epoch 1/1000, Step: 0/1, total loss:0.0530
Epoch 2/1000, Step: 0/1, total loss:0.0446
Epoch 3/1000, Step: 0/1, total loss:0.0415


# 模型使用

In [ ]:
# 将模型调成推理模式
device = 'cpu'
model = model.eval()
model = model.to(device)

In [ ]:
text = "本人账号被盗，资金被江西（杨建）挪用，请亚马逊尽快查实，将本人的200元资金退回。本人已于2017年11月30日提交退货申请，为何到2018年了还是没解决？亚马逊是什么情况？请给本人一个合理解释。"

In [ ]:
def predict(text):
    """
    模型推理，输入为评论，输出为摘要
    :param text: 一个长评论
    :return: 短摘要
    """
    # 对长评论进行分词
    src = tokenizer(text, return_tensors='pt')
    # 构造[CLS]，即开始标志
    tgt = {
        'input_ids': torch.LongTensor([[101]]), # 101为开始标志
        'attention_mask': torch.LongTensor([[1]]),
    }
    src, tgt = to_device(src), to_device(tgt)

    # 循环反复调用模型进行推理，直到达到摘要最大长度或遇到结束(102)标志
    for i in range(summary_max_length):
        outputs = model(src, tgt)
        index = model.predictor(outputs[:, -1, :]).argmax()
        tgt['input_ids'] = torch.concat([tgt['input_ids'], index.view(1, -1)], dim=1)
        tgt['attention_mask'] = torch.concat([tgt['attention_mask'], torch.LongTensor([[1]])], dim=1)

        if index == 102:
            break

    tokens = tokenizer.convert_ids_to_tokens(tgt['input_ids'].squeeze())
    return ''.join(tokens).replace("[CLS]", "").replace("[SEP]", "")

In [ ]:
predict(text)